<center><h1>Pranav Pandy Mohanapandian_HW7</h1></center>

Name: Pranav Pandy Mohanapandian<br>Github Username: pranav-pandy<br>USC ID: 6381490053

## 1. Multi-class and Multi-Label Classification Using Support Vector Machines

### Import packages

In [8]:
import numpy as np
import pandas as pd
import random

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix, f1_score, hamming_loss, silhouette_samples
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC

import warnings
warnings.filterwarnings("ignore")

### (a) Download the Anuran Calls (MFCCs) Data Set

In [13]:
anuran_data = pd.read_csv('../data/Anuran Calls (MFCCs)/Frogs_MFCCs.csv')
anuran_data


,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [15]:
total_samples = len(anuran_data)
random.seed(total_samples)
train_size = int(total_samples * 0.7)
train_indices = random.sample(range(total_samples), train_size)

test_indices = [idx for idx in range(total_samples) if idx not in train_indices]

data_train = anuran_data.iloc[train_indices, :].reset_index(drop=True)
data_test = anuran_data.iloc[test_indices, :].reset_index(drop=True)

features_train = data_train.iloc[:, :-4]
labels_family_train = data_train['Family']
labels_genus_train = data_train['Genus']
labels_species_train = data_train['Species']

features_test = data_test.iloc[:, :-4]
labels_family_test = data_test['Family']
labels_genus_test = data_test['Genus']
labels_species_test = data_test['Species']


### (b) Train a classifier for each label

#### (i) Research

1. **What is Exact Match?**

    * Exact Match is a metric in multilabel classification that checks whether every label for a sample is correctly predicted.
    * Specifically, it calculates the percentage of samples where the predicted label set exactly equals the true label set.
    * Partial matches don’t count—it’s all or nothing.

2. **What is Hamming Loss?**

    * Hamming Loss quantifies how often labels are predicted incorrectly in a multilabel setting.

    * It’s computed as the average number of misclassified labels per instance, considering both false positives and false negatives across all label categories.

#### (ii) Train a SVM for each of the labels

In [21]:
def evaluate_multilabel(true_labels, predicted_labels):
    total_errors = 0
    for actual, predicted in zip(true_labels.values, predicted_labels.values):
        total_errors += np.sum(actual != predicted)

    total_labels = true_labels.shape[0] * true_labels.shape[1]
    hamming_loss_value = total_errors / total_labels

    exact_matches = 0
    for actual, predicted in zip(true_labels.values, predicted_labels.values):
        if np.sum(actual == predicted) == true_labels.shape[1]:
            exact_matches += 1
    exact_match_score = exact_matches / true_labels.shape[0]

    hamming_loss_value = np.round(hamming_loss_value, 4)
    exact_match_score = np.round(exact_match_score, 4)

    print("Hamming Loss:", hamming_loss_value)
    print("Exact Match Score:", exact_match_score)

    return [hamming_loss_value, exact_match_score]


In [23]:
def assess_models(test_features, test_labels, trained_models):
    predictions = pd.DataFrame(columns=test_labels.columns)
    
    for tag in test_labels.columns:
        clf = trained_models[tag]
        pred_values = clf.predict(test_features)
        predictions.loc[:, tag] = pred_values

    return evaluate_multilabel(test_labels, predictions)


In [25]:
def tune_model(base_clf, grid_args, train_x, train_y, test_x, test_y):
    search = GridSearchCV(estimator=base_clf, **grid_args, n_jobs=-1)
    search.fit(train_x, train_y)

    param_summary = {}
    for entry in search.cv_results_['params']:
        for param_name, param_val in entry.items():
            param_summary.setdefault(param_name, []).append(param_val)

    mean_scores = np.round(np.array(search.cv_results_['mean_test_score']), 5)
    param_summary['mean_test_score'] = mean_scores

    result_df = pd.DataFrame(param_summary)
    display(result_df)

    print("\nOptimal Parameters:")
    print(search.best_params_, "\n")
    print(classification_report(test_y, search.predict(test_x)))

    return search


In [27]:
svc_models_raw = {}
results_overview = {}

grid_options = {
    'scoring': 'f1_weighted',
    'param_grid': {
        'C': np.logspace(1, 4, 4),
        'gamma': np.logspace(-3, 6, 10)
    },
    'cv': StratifiedKFold(n_splits=10, shuffle=True, random_state=69)
}

experiment_title = "RBF SVM without Feature Scaling"


In [29]:
raw_kernel_models = {}
model_results = {}

search_params = {
    'scoring': 'f1_weighted',
    'param_grid': {
        'C': np.logspace(1, 4, 4),
        'gamma': np.logspace(-3, 6, 10)
    },
    'cv': StratifiedKFold(n_splits=10, shuffle=True, random_state=69)
}

run_description = "Unscaled RBF Kernel SVM"


In [33]:
rbf_svm_models_unscaled = {}
evaluation_table = {}

grid_config = {
    'scoring': 'f1_weighted',
    'param_grid': {
        'C': np.logspace(1, 4, 4),
        'gamma': np.logspace(-3, 6, 10)
    },
    'cv': StratifiedKFold(n_splits=10, shuffle=True, random_state=69)
}

description_note = "RBF Support Vector Machine (No Standardization)"


In [35]:
print("Target Label: Genus")
rbf_svm_models_unscaled['Genus'] = tune_model(
    SVC(kernel='rbf'),
    grid_config,
    features_train,
    labels_genus_train,
    features_test,
    labels_genus_test
)


Target Label: Genus


,C,gamma,mean_test_score
0,10.0,0.001,0.74498
1,10.0,0.010,0.91808
2,10.0,0.100,0.97195
3,10.0,1.000,0.98781
4,10.0,10.000,0.98021
5,10.0,100.000,0.73350
6,10.0,1000.000,0.44239
7,10.0,10000.000,0.42458
8,10.0,100000.000,0.42324
9,10.0,1000000.000,0.42324



Optimal Parameters:
{'C': 100.0, 'gamma': 1.0} 

               precision    recall  f1-score   support

    Adenomera       0.99      1.00      1.00      1240
     Ameerega       0.99      0.99      0.99       154
Dendropsophus       0.97      0.95      0.96        91
    Hypsiboas       0.99      0.99      0.99       479
Leptodactylus       0.95      1.00      0.97        77
Osteocephalus       1.00      0.87      0.93        47
     Rhinella       1.00      0.93      0.97        15
       Scinax       1.00      0.98      0.99        56

     accuracy                           0.99      2159
    macro avg       0.99      0.96      0.97      2159
 weighted avg       0.99      0.99      0.99      2159



In [39]:
print("Evaluation Summary:", run_description)
model_results[run_description] = assess_models(
    features_test,
    data_test[['Genus']],  # Only evaluating Genus since only that model exists
    {'Genus': rbf_svm_models_unscaled['Genus']}
)



Evaluation Summary: Unscaled RBF Kernel SVM
Hamming Loss: 0.0102
Exact Match Score: 0.9898


In [41]:
scaler_obj = StandardScaler()
train_scaled = scaler_obj.fit_transform(features_train)
test_scaled = scaler_obj.fit_transform(features_test)

rbf_svm_models_scaled = {}
run_description = "RBF SVM with Normalized Features"


In [43]:
print("Method:", run_description)
print("Target Label: Family")

rbf_svm_models_scaled['Family'] = tune_model(
    SVC(kernel='rbf'),
    grid_config,
    train_scaled,
    labels_family_train,
    test_scaled,
    labels_family_test
)


Method: RBF SVM with Normalized Features
Target Label: Family


,C,gamma,mean_test_score
0,10.0,0.001,0.94461
1,10.0,0.010,0.98726
2,10.0,0.100,0.98944
3,10.0,1.000,0.90614
4,10.0,10.000,0.57518
5,10.0,100.000,0.47315
6,10.0,1000.000,0.46983
7,10.0,10000.000,0.46983
8,10.0,100000.000,0.46983
9,10.0,1000000.000,0.46983



Optimal Parameters:
{'C': 100.0, 'gamma': 0.01} 

                 precision    recall  f1-score   support

      Bufonidae       0.78      0.93      0.85        15
  Dendrobatidae       0.99      0.99      0.99       154
        Hylidae       0.99      0.98      0.98       673
Leptodactylidae       0.99      0.99      0.99      1317

       accuracy                           0.99      2159
      macro avg       0.94      0.97      0.95      2159
   weighted avg       0.99      0.99      0.99      2159



In [45]:
print("Target Label: Genus")

rbf_svm_models_scaled['Genus'] = tune_model(
    SVC(kernel='rbf'),
    grid_config,
    train_scaled,
    labels_genus_train,
    test_scaled,
    labels_genus_test
)


Target Label: Genus


,C,gamma,mean_test_score
0,10.0,0.001,0.95753
1,10.0,0.010,0.98518
2,10.0,0.100,0.98739
3,10.0,1.000,0.85510
4,10.0,10.000,0.51234
5,10.0,100.000,0.42645
6,10.0,1000.000,0.42324
7,10.0,10000.000,0.42324
8,10.0,100000.000,0.42324
9,10.0,1000000.000,0.42324



Optimal Parameters:
{'C': 10.0, 'gamma': 0.1} 

               precision    recall  f1-score   support

    Adenomera       0.99      1.00      1.00      1240
     Ameerega       1.00      1.00      1.00       154
Dendropsophus       1.00      0.95      0.97        91
    Hypsiboas       0.96      1.00      0.98       479
Leptodactylus       1.00      0.97      0.99        77
Osteocephalus       1.00      0.79      0.88        47
     Rhinella       1.00      0.87      0.93        15
       Scinax       1.00      0.95      0.97        56

     accuracy                           0.99      2159
    macro avg       0.99      0.94      0.96      2159
 weighted avg       0.99      0.99      0.99      2159



In [47]:
print("Target Label: Species")

rbf_svm_models_scaled['Species'] = tune_model(
    SVC(kernel='rbf'),
    grid_config,
    train_scaled,
    labels_species_train,
    test_scaled,
    labels_species_test
)


Target Label: Species


,C,gamma,mean_test_score
0,10.0,0.001,0.96716
1,10.0,0.010,0.98659
2,10.0,0.100,0.98474
3,10.0,1.000,0.83209
4,10.0,10.000,0.38807
5,10.0,100.000,0.31523
6,10.0,1000.000,0.31194
7,10.0,10000.000,0.31194
8,10.0,100000.000,0.31194
9,10.0,1000000.000,0.31194



Optimal Parameters:
{'C': 1000.0, 'gamma': 0.01} 

                        precision    recall  f1-score   support

        AdenomeraAndre       0.98      0.98      0.98       182
AdenomeraHylaedactylus       0.99      1.00      1.00      1058
    Ameeregatrivittata       0.99      0.99      0.99       154
            HylaMinuta       0.94      0.93      0.94        91
  HypsiboasCinerascens       0.97      0.99      0.98       153
     HypsiboasCordobae       0.99      0.98      0.99       326
   LeptodactylusFuscus       0.97      0.99      0.98        77
 OsteocephalusOophagus       0.95      0.87      0.91        47
     Rhinellagranulosa       0.83      1.00      0.91        15
           ScinaxRuber       1.00      0.98      0.99        56

              accuracy                           0.99      2159
             macro avg       0.96      0.97      0.97      2159
          weighted avg       0.99      0.99      0.99      2159



In [49]:
print("Evaluation Summary:", run_description)

model_results[run_description] = assess_models(
    test_scaled,
    data_test.iloc[:, -4:-1],
    rbf_svm_models_scaled
)


Evaluation Summary: RBF SVM with Normalized Features
Hamming Loss: 0.0127
Exact Match Score: 0.9764


#### (iii) Repeat 1(b)ii with L1-penalized SVMs

In [51]:
l1_svm_models = {}
grid_config_l1 = {
    'scoring': 'f1_weighted',
    'param_grid': {'C': np.logspace(1, 5, 10)},
    'cv': StratifiedKFold(n_splits=10, shuffle=True, random_state=69)
}
run_description = "L1-Regularized Linear SVM"


In [53]:
print("Method:", run_description)
print("Target Label: Family")

l1_svm_models['Family'] = tune_model(
    LinearSVC(penalty='l1', dual=False, max_iter=10000),
    grid_config_l1,
    train_scaled,
    labels_family_train,
    test_scaled,
    labels_family_test
)


Method: L1-Regularized Linear SVM
Target Label: Family


,C,mean_test_score
0,10.000000,0.92847
1,27.825594,0.92866
2,77.426368,0.92866
3,215.443469,0.92866
4,599.484250,0.92866
5,1668.100537,0.92866
6,4641.588834,0.92866
7,12915.496650,0.92866
8,35938.136638,0.92866
9,100000.000000,0.92866



Optimal Parameters:
{'C': 27.825594022071243} 

                 precision    recall  f1-score   support

      Bufonidae       0.00      0.00      0.00        15
  Dendrobatidae       0.88      0.92      0.90       154
        Hylidae       0.94      0.90      0.92       673
Leptodactylidae       0.95      0.98      0.96      1317

       accuracy                           0.94      2159
      macro avg       0.69      0.70      0.70      2159
   weighted avg       0.94      0.94      0.94      2159



In [55]:
print("Target Label: Genus")

l1_svm_models['Genus'] = tune_model(
    LinearSVC(penalty='l1', dual=False, max_iter=10000),
    grid_config_l1,
    train_scaled,
    labels_genus_train,
    test_scaled,
    labels_genus_test
)


Target Label: Genus


,C,mean_test_score
0,10.000000,0.94693
1,27.825594,0.94716
2,77.426368,0.94716
3,215.443469,0.94716
4,599.484250,0.94716
5,1668.100537,0.94697
6,4641.588834,0.94697
7,12915.496650,0.94697
8,35938.136638,0.94697
9,100000.000000,0.94697



Optimal Parameters:
{'C': 27.825594022071243} 

               precision    recall  f1-score   support

    Adenomera       0.96      0.99      0.98      1240
     Ameerega       0.92      0.95      0.93       154
Dendropsophus       0.92      0.67      0.78        91
    Hypsiboas       0.92      0.98      0.95       479
Leptodactylus       0.97      0.90      0.93        77
Osteocephalus       1.00      0.36      0.53        47
     Rhinella       0.91      0.67      0.77        15
       Scinax       0.95      0.93      0.94        56

     accuracy                           0.95      2159
    macro avg       0.94      0.80      0.85      2159
 weighted avg       0.95      0.95      0.94      2159



In [57]:
print("Target Label: Species")

l1_svm_models['Species'] = tune_model(
    LinearSVC(penalty='l1', dual=False, max_iter=10000),
    grid_config_l1,
    train_scaled,
    labels_species_train,
    test_scaled,
    labels_species_test
)


Target Label: Species


,C,mean_test_score
0,10.000000,0.95877
1,27.825594,0.95857
2,77.426368,0.95837
3,215.443469,0.95837
4,599.484250,0.95837
5,1668.100537,0.95837
6,4641.588834,0.95837
7,12915.496650,0.95837
8,35938.136638,0.95837
9,100000.000000,0.95837



Optimal Parameters:
{'C': 10.0} 

                        precision    recall  f1-score   support

        AdenomeraAndre       0.88      0.94      0.91       182
AdenomeraHylaedactylus       0.99      1.00      0.99      1058
    Ameeregatrivittata       0.94      0.94      0.94       154
            HylaMinuta       0.91      0.70      0.80        91
  HypsiboasCinerascens       0.92      0.94      0.93       153
     HypsiboasCordobae       0.92      0.96      0.94       326
   LeptodactylusFuscus       0.97      0.91      0.94        77
 OsteocephalusOophagus       1.00      0.49      0.66        47
     Rhinellagranulosa       0.72      0.87      0.79        15
           ScinaxRuber       0.93      0.96      0.95        56

              accuracy                           0.95      2159
             macro avg       0.92      0.87      0.88      2159
          weighted avg       0.95      0.95      0.95      2159



In [59]:
print("Evaluation Summary:", run_description)

model_results[run_description] = assess_models(
    test_scaled,
    data_test.iloc[:, -4:-1],
    l1_svm_models
)


Evaluation Summary: L1-Regularized Linear SVM
Hamming Loss: 0.0523
Exact Match Score: 0.9143


#### (iv) Repeat 1(b)iii by using SMOTE or any other method for imbalance

In [61]:
def tune_with_smote(base_clf, grid_settings, train_x, train_y, test_x, test_y):
    smote_pipeline = Pipeline([
        ('oversample', SMOTE()),
        ('model', base_clf)
    ])
    optimized_model = tune_model(smote_pipeline, grid_settings, train_x, train_y, test_x, test_y)
    return optimized_model


In [63]:
smote_l1_models = {}
grid_config_smote = {
    'scoring': 'f1_weighted',
    'param_grid': {'model__C': np.logspace(1, 5, 10)},
    'cv': StratifiedKFold(n_splits=10, shuffle=True, random_state=69)
}
run_description = "Linear SVM with L1 Regularization and SMOTE"


In [65]:
print("Method:", run_description)
print("Target Label: Family")

smote_l1_models['Family'] = tune_with_smote(
    LinearSVC(penalty='l1', dual=False, max_iter=20000),
    grid_config_smote,
    train_scaled,
    labels_family_train,
    test_scaled,
    labels_family_test
)


Method: Linear SVM with L1 Regularization and SMOTE
Target Label: Family


,model__C,mean_test_score
0,10.000000,0.92066
1,27.825594,0.92170
2,77.426368,0.92022
3,215.443469,0.92073
4,599.484250,0.92028
5,1668.100537,0.91954
6,4641.588834,0.92102
7,12915.496650,0.92036
8,35938.136638,0.92136
9,100000.000000,0.92104



Optimal Parameters:
{'model__C': 27.825594022071243} 

                 precision    recall  f1-score   support

      Bufonidae       0.22      0.93      0.36        15
  Dendrobatidae       0.78      0.97      0.86       154
        Hylidae       0.95      0.86      0.90       673
Leptodactylidae       0.96      0.95      0.95      1317

       accuracy                           0.92      2159
      macro avg       0.73      0.93      0.77      2159
   weighted avg       0.94      0.92      0.93      2159



In [67]:
print("Target Label: Genus")

smote_l1_models['Genus'] = tune_with_smote(
    LinearSVC(penalty='l1', dual=False, max_iter=10000),
    grid_config_smote,
    train_scaled,
    labels_genus_train,
    test_scaled,
    labels_genus_test
)


Target Label: Genus


,model__C,mean_test_score
0,10.000000,0.92048
1,27.825594,0.91846
2,77.426368,0.91891
3,215.443469,0.91797
4,599.484250,0.92049
5,1668.100537,0.91915
6,4641.588834,0.92025
7,12915.496650,0.91675
8,35938.136638,0.91904
9,100000.000000,0.91873



Optimal Parameters:
{'model__C': 599.4842503189409} 

               precision    recall  f1-score   support

    Adenomera       0.99      0.92      0.95      1240
     Ameerega       0.82      0.91      0.86       154
Dendropsophus       0.64      0.95      0.76        91
    Hypsiboas       0.97      0.90      0.94       479
Leptodactylus       0.93      0.91      0.92        77
Osteocephalus       0.58      0.77      0.66        47
     Rhinella       0.24      0.87      0.38        15
       Scinax       0.84      0.96      0.90        56

     accuracy                           0.91      2159
    macro avg       0.75      0.90      0.80      2159
 weighted avg       0.94      0.91      0.92      2159



In [69]:
print("Target Label: Species")

smote_l1_models['Species'] = tune_with_smote(
    LinearSVC(penalty='l1', dual=False, max_iter=25000),
    grid_config_smote,
    train_scaled,
    labels_species_train,
    test_scaled,
    labels_species_test
)


Target Label: Species


,model__C,mean_test_score
0,10.000000,0.95744
1,27.825594,0.95782
2,77.426368,0.95742
3,215.443469,0.95824
4,599.484250,0.95939
5,1668.100537,0.95757
6,4641.588834,0.95825
7,12915.496650,0.95927
8,35938.136638,0.95809
9,100000.000000,0.95928



Optimal Parameters:
{'model__C': 599.4842503189409} 

                        precision    recall  f1-score   support

        AdenomeraAndre       0.95      0.92      0.94       182
AdenomeraHylaedactylus       0.99      0.99      0.99      1058
    Ameeregatrivittata       0.94      0.88      0.91       154
            HylaMinuta       0.81      0.91      0.86        91
  HypsiboasCinerascens       0.94      0.92      0.93       153
     HypsiboasCordobae       0.95      0.91      0.93       326
   LeptodactylusFuscus       0.91      0.90      0.90        77
 OsteocephalusOophagus       0.79      0.72      0.76        47
     Rhinellagranulosa       0.36      0.93      0.52        15
           ScinaxRuber       0.90      0.96      0.93        56

              accuracy                           0.95      2159
             macro avg       0.86      0.91      0.87      2159
          weighted avg       0.96      0.95      0.95      2159



In [70]:
print("Evaluation Summary:", run_description)

model_results[run_description] = assess_models(
    test_scaled,
    data_test.iloc[:, -4:-1],
    smote_l1_models
)


Evaluation Summary: Linear SVM with L1 Regularization and SMOTE
Hamming Loss: 0.0723
Exact Match Score: 0.8536


In [71]:
evaluation_df = pd.DataFrame(data=model_results)
evaluation_df.index = ['Hamming Loss', 'Exact Match Score']
evaluation_df


,Unscaled RBF Kernel SVM,RBF SVM with Normalized Features,L1-Regularized Linear SVM,Linear SVM with L1 Regularization and SMOTE
Hamming Loss,0.0102,0.0127,0.0523,0.0723
Exact Match Score,0.9898,0.9764,0.9143,0.8536


### Observations
    * The Gaussian Kernel SVM achieved the best exact match performance among all models.

    * The L1-regularized SVM showed a drop in exact match ratio and an increase in hamming loss compared to the Gaussian SVM.

    * Applying SMOTE to the L1 SVM helped improve the exact match ratio, indicating that balancing class distributions had a positive effect.



### Conclusion
    * Based on both exact match and hamming loss, the Gaussian Kernel SVM consistently outperforms the other models, indicating it is the most effective approach for this multi-label classification task.



## 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

### (a) & (b) Use k-means clustering & Determine which family is the majority

In [79]:
def select_optimal_clusters(max_clusters, features, seed_val):
    optimal_k = 2
    highest_silhouette = 0

    for k in range(2, max_clusters + 1):
        kmeans_model = KMeans(n_clusters=k, random_state=seed_val)
        predicted_clusters = kmeans_model.fit_predict(features)
        silhouette_val = silhouette_score(features, predicted_clusters)

        if silhouette_val > highest_silhouette:
            optimal_k = k
            highest_silhouette = silhouette_val

    print("Selected number of clusters:", optimal_k)
    return optimal_k


In [81]:
def compute_cluster_modes(k_val, assignments, true_labels):
    label_lookup = pd.DataFrame(columns=true_labels.columns)

    for cluster_id in range(k_val):
        points_in_cluster = np.where(assignments == cluster_id)[0]
        cluster_subset = true_labels.iloc[points_in_cluster, :]

        dominant_labels = []
        for col in true_labels.columns:
            most_common = cluster_subset[col].value_counts().idxmax()
            dominant_labels.append(most_common)

        label_lookup.loc[cluster_id] = dominant_labels

    return label_lookup


In [83]:
def assess_clustering(label_map, cluster_assignments, ground_truth):
    total_mismatches = 0

    for cluster_id in range(len(label_map)):
        member_indices = np.where(cluster_assignments == cluster_id)[0]
        true_group = ground_truth.iloc[member_indices].values
        predicted_group = label_map.loc[cluster_id].values

        for actual in true_group:
            total_mismatches += np.sum(actual != predicted_group)

    hamming_distance = total_mismatches / ground_truth.shape[0]
    hamming_loss_val = total_mismatches / (ground_truth.shape[0] * ground_truth.shape[1])

    return hamming_distance, hamming_loss_val


In [85]:
def run_monte_carlo(iterations, features, true_labels):
    dist_scores = []
    loss_scores = []

    for run in range(iterations):
        print(f"Iteration: {run + 1}")

        k = select_optimal_clusters(iterations, features, run)
        km_model = KMeans(n_clusters=k, random_state=run)
        cluster_assignments = km_model.fit_predict(features)

        label_reference = compute_cluster_modes(k, cluster_assignments, true_labels)
        display(label_reference)

        dist, loss = assess_clustering(label_reference, cluster_assignments, true_labels)
        dist_scores.append(dist)
        loss_scores.append(loss)

        print(f"Hamming Distance: {round(dist, 4)}")
        print(f"Hamming Loss: {round(loss, 4)}\n")

    return dist_scores, loss_scores


In [87]:
distances, losses = run_monte_carlo(50, anuran_data.iloc[:, :-4], anuran_data.iloc[:, -4:-1])


Iteration: 1
Selected number of clusters: 4


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Hylidae,Hypsiboas,HypsiboasCinerascens
3,Leptodactylidae,Adenomera,AdenomeraAndre


Hamming Distance: 0.7022
Hamming Loss: 0.2341

Iteration: 2
Selected number of clusters: 4


,Family,Genus,Species
0,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Leptodactylidae,Adenomera,AdenomeraAndre
3,Hylidae,Hypsiboas,HypsiboasCordobae


Hamming Distance: 0.7358
Hamming Loss: 0.2453

Iteration: 3
Selected number of clusters: 4


...

Hamming Distance: 0.6673
Hamming Loss: 0.2224

Iteration: 49
Selected number of clusters: 3


,Family,Genus,Species
0,Dendrobatidae,Ameerega,Ameeregatrivittata
1,Leptodactylidae,Adenomera,AdenomeraHylaedactylus
2,Hylidae,Hypsiboas,HypsiboasCordobae


Hamming Distance: 0.7155
Hamming Loss: 0.2385

Iteration: 50
Selected number of clusters: 3


,Family,Genus,Species
0,Hylidae,Hypsiboas,HypsiboasCordobae
1,Hylidae,Hypsiboas,HypsiboasCordobae
2,Leptodactylidae,Adenomera,AdenomeraHylaedactylus


Hamming Distance: 0.8905
Hamming Loss: 0.2968



### (c) Calculate the average Hamming distance, Hamming score, and Hamming 

In [92]:
print("Average Hamming Distance:", round(np.mean(distances), 5))
print("Average Hamming Loss:", round(np.mean(losses), 5))
print("Average Hamming Score:", round(1 - np.mean(losses), 5))


Average Hamming Distance: 0.71867
Average Hamming Loss: 0.23956
Average Hamming Score: 0.76044


## 3. ISLR 12.6.2

### Q3 - Parts A and B
![Q3 A and B](../3%20Ques%20A%20and%20B.jpg)

---

### Q3 - Parts C, D, and E
![Q3 C, D, and E](../3%20Ques%20C%2CD%2CE.jpg)


### Q3 - Parts A and B  
![3 Ques A and B.jpg](attachment:3%20Ques%20A%20and%20B.jpg)

---

### Q3 - Parts C, D, and E  
![3 Ques C,D,E.jpg](attachment:3%20Ques%20C%2CD%2CE.jpg)
